In [211]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv(r'C:\Users\USER\Documents\git\temp\경기도경기부동산포털토지실거래가정보.csv', encoding='cp949')

C:\Users\USER\AppData\Local\Temp\ipykernel_23912\1658827424.py:1: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(r'C:\Users\USER\Documents\git\temp\경기도경기부동산포털토지실거래가정보.csv', encoding='cp949')


# 1

In [34]:
data.columns

Index(['시군구코드', '접수년도', '처리구분', '접수번호', '물건번호', '상세물건번호', '변경구분', '공개반영일',
       '읍면동코드', '시군구명', '읍면동리명', '용도지역코드', '용도지역명', '지목코드', '지목명', '계약일',
       '토지거래면적', '지분거래구분', '거래금액', '해제여부', '해제사유', '해제사유발생일', '중개구분', '중개업소주소',
       '신고번호', '물건번호.1'],
      dtype='object')

# 2

In [32]:
data.loc[:, data.isna().sum() != 0].isna().sum().sort_values()

용도지역명          4
용도지역코드         9
중개구분         236
변경구분       16265
공개반영일      16265
중개업소주소     48522
해제사유발생일    49191
dtype: int64

# 3

In [22]:
data.isna().sum() - data.query('변경구분 != 변경구분').isna().sum()

시군구코드          0
접수년도           0
처리구분           0
접수번호           0
물건번호           0
상세물건번호         0
변경구분           0
공개반영일          0
읍면동코드          0
시군구명           0
읍면동리명          0
용도지역코드         4
용도지역명          4
지목코드           0
지목명            0
계약일            0
토지거래면적         0
지분거래구분         0
거래금액           0
해제여부           0
해제사유           0
해제사유발생일    33624
중개구분         129
중개업소주소     36681
신고번호           0
물건번호.1         0
dtype: int64

# 4

In [44]:
data = data.loc[:, data.isna().sum() < 300].dropna()
data.shape

(52702, 22)

# 5

In [68]:
(
    data.filter(items=['지목명', '중개구분', '거래금액'])
    .groupby(['지목명', '중개구분']).mean()
    .reset_index()
    .pivot(index=['중개구분'], columns=['지목명'], values=['거래금액'])
    .diff().abs().iloc[1:]
    .T
    .sort_values(by=['직거래'])
)

중개구분                  직거래
     지목명                 
거래금액 제방        189.692075
     잡종지       251.537567
     도로        492.342967
     염전       1864.752874
     수도용지     2743.119048
     하천       5450.341608
     구거       7055.533333
     주유소용지    7197.660256
     주차장      7293.577443
     답       10184.591666
     전       10801.453766
     창고용지    12309.788122
     묘지      12702.324740
     과수원     14789.556667
     임야      20415.617734
     목장용지    20958.328423
     유지      23020.035256
     유원지     24586.175000
     공장용지    24891.579687
     대       26357.556049
     공원      27328.329787
     양어장     29134.020000
     종교용지    33142.625000
     철도용지    40140.888889
     체육용지   172109.339422
     학교용지             NaN

# 6

In [116]:
data.신고번호.value_counts().head(5)

신고번호
78151505     76
14281996     76
122227421    75
61458212     73
22886087     71
Name: count, dtype: int64

In [125]:
max_ = data.query('신고번호==122227421').토지거래면적.max()
max_

524534

In [126]:
data.query(f'신고번호==122227421 and 토지거래면적=={max_}')

,시군구코드,접수년도,처리구분,접수번호,물건번호,상세물건번호,읍면동코드,시군구명,읍면동리명,용도지역코드,...,지목명,계약일,토지거래면적,지분거래구분,거래금액,해제여부,해제사유,중개구분,신고번호,물건번호.1
48398,41670,2024,4,2439,73,1,4167025035,경기도여주시,가남읍 양귀리,UQB100,...,체육용지,20240214,524534,전체거래,9756347,2,추가입력완료,직거래,122227421,73


# 7

In [161]:
(
    data.groupby(['접수번호', '신고번호']).count()
    .iloc[:, :1]
    .sort_values(by='시군구코드', ascending=False)
    .reset_index()
    .접수번호.value_counts()
    .to_frame()
    .query('count >= 2')
)

,count
접수번호,
1534,13
961,12
1514,12
1524,11
1647,11
...,...
9589,2
7286,2
7276,2


# 8

In [279]:
target_dates = data.계약일.value_counts().head(5).index

In [290]:
temp = data.query('계약일 in @target_dates and 토지거래면적 < 1000')
target_land_category = temp.지목명.value_counts().index[:5]
temp.query('지목명 in @target_land_category').filter(items=['지목명', '거래금액']).groupby(['지목명']).mean().sort_values(by=['거래금액'], ascending=False)

,거래금액
지목명,
대,21580.187192
전,10693.898964
답,9162.932692
임야,5540.376991
도로,1047.071556


# 9

In [267]:
(
    data.filter(items=['시군구명', '거래금액'])
    .merge(data.filter(items=['시군구명', '거래금액']).groupby(['시군구명']).mean().rename(columns={'거래금액': '평균거래금액'}).reset_index())
    .query('거래금액 > 평균거래금액')
    .drop(columns=['평균거래금액'])
    .groupby(['시군구명']).mean()
    .sort_values(by=['거래금액'], ascending=False)
    .diff().abs()
)

,거래금액
시군구명,
경기도 성남시수정구,NaN
경기도구리시,362367.33333
경기도안양시동안구,15446251.16667
경기도 오산시,41646.19231
경기도부천시오정구,82044.39860
...,...
경기도 양평군,378.85097
경기도연천군,2251.04798
경기도 연천군,5197.48616


In [248]:
data.filter(items=['지목명', '토지거래면적']).groupby(['지목명']).mean().sort_values(by=['토지거래면적'], ascending=False)

,토지거래면적
지목명,
체육용지,10210.951220
학교용지,5435.166667
임야,1423.680316
답,1238.822792
창고용지,936.804348
공장용지,934.679644
주차장,931.629213
유지,883.560000
목장용지,794.443299


# 10

In [241]:
temp = data.query('토지거래면적 < 500')
temp = temp.loc[np.where(temp.용도지역명.str.extract('(주거)', expand=False).isna(), False, True)]
temp['year'] = temp.계약일.apply(lambda x: str(x)[:4])
temp.filter(items=['year', '시군구명', '거래금액']).groupby(['year', '시군구명']).mean().reset_index().groupby(['year']).max()

,시군구명,거래금액
year,,
2021,경기도용인시기흥구,49304.0
2022,경기도수원시팔달구,292300.0
2023,경기도화성시,69300.0
2024,경기도화성시,230000.0
